In [1]:
import os
import pandas as pd
import surprise

In [3]:
df = pd.read_csv(r"C:\Users\solharsh\Desktop\Dataset\rating_final.csv")

In [4]:
df.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [15]:
df.head()
df.rename(columns={"userID":"user","placeID":"item","rating":"rating"})

,user,item,rating
0,U1077,135085,2
1,U1077,135038,2
2,U1077,132825,2
3,U1077,135060,1
4,U1068,135104,1
...,...,...,...
1156,U1043,132630,1
1157,U1011,132715,1
1158,U1068,132733,1
1159,U1068,132594,1


In [18]:
#The column name must be user item and rating. Surprise only accepts 3 columns

reader = surprise.dataset.Reader(line_format='user item rating',rating_scale=(1,5))

In [88]:
data = surprise.dataset.Dataset.load_from_df(df,reader=reader)
# Retrieve the trainset.
trainset = data.build_full_trainset()

In [20]:
data.raw_rating #list of tuples containing the data in different format

[('U1077', 135085, 2.0, None),
 ('U1077', 135038, 2.0, None),
 ('U1077', 132825, 2.0, None),
 ('U1077', 135060, 1.0, None),
 ('U1068', 135104, 1.0, None),
 ('U1068', 132740, 0.0, None),
 ('U1068', 132663, 1.0, None),
 ('U1068', 132732, 0.0, None),
 ('U1068', 132630, 1.0, None),
 ('U1067', 132584, 2.0, None),
 ('U1067', 132733, 1.0, None),
 ('U1067', 132732, 1.0, None),
 ('U1067', 132630, 1.0, None),
 ('U1067', 135104, 0.0, None),
 ('U1067', 132560, 1.0, None),
 ('U1103', 132584, 1.0, None),
 ('U1103', 132732, 0.0, None),
 ('U1103', 132630, 1.0, None),
 ('U1103', 132613, 2.0, None),
 ('U1103', 132667, 1.0, None),
 ('U1103', 135104, 1.0, None),
 ('U1103', 132663, 1.0, None),
 ('U1103', 132733, 2.0, None),
 ('U1107', 132660, 2.0, None),
 ('U1107', 132584, 2.0, None),
 ('U1107', 132733, 2.0, None),
 ('U1044', 135088, 2.0, None),
 ('U1044', 132583, 1.0, None),
 ('U1070', 132608, 2.0, None),
 ('U1070', 132609, 1.0, None),
 ('U1070', 132613, 1.0, None),
 ('U1031', 132663, 0.0, None),
 ('U1031

In [22]:
#we can read via a text file too but the columns should be in that format mentioned above
reader = surprise.dataset.Reader(line_format='user rating item',sep=",",rating_scale=(1,5),skip_lines=1)

In [23]:
#data1 = surprise.dataset.Dataset.load_from_file("sample.csv",reader=reader)

In [25]:
data = data.build_full_trainset()

In [26]:
## create a neighbourhood based uder and item based collaborative filtering model

In [32]:
import surprise.prediction_algorithms.knns as knns #knns = builds basic collaborative filtering model
knnbasic = knns.KNNBasic(k=40,min_k=1,sims_options={'name':'cosine','user_based':True}) #k = no of neighbours considered for 
#closeness of a user or an item #sims_options = similarity options

In [36]:
knnbasic.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [38]:
#once this model is trained, we can use it to predict rating of a certain user or item

In [40]:
df.head()

,userID,placeID,rating
0,U1077,135085,2
1,U1077,135038,2
2,U1077,132825,2
3,U1077,135060,1
4,U1068,135104,1


In [51]:
knnbasic.predict(uid='U1077',iid=135085,r_ui=2)
#if we will see that the predicted rating of 1.60 is not too far from the actual value of 2

Prediction(uid='U1077', iid=135085, r_ui=2, est=1.6019471488178028, details={'actual_k': 36, 'was_impossible': False})

In [52]:
#item based collaborative filter

In [53]:
knnbasic = knns.KNNBasic(k=40,min_k=1,sims_options={'name':'cosine','user_based':False})

In [54]:
knnbasic.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [55]:
knnbasic.predict(uid='U1077',iid=135038)

Prediction(uid='U1077', iid=135038, r_ui=None, est=1.490206957776175, details={'actual_k': 24, 'was_impossible': False})

In [56]:
## collaborative filter with average effects

In [58]:
knnbasic = knns.KNNBasic(k=40,min_k=1,sims_options={'name':'pearson','user_based':False})

In [60]:
knnbasic.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [61]:
df.head()

,userID,placeID,rating
0,U1077,135085,2
1,U1077,135038,2
2,U1077,132825,2
3,U1077,135060,1
4,U1068,135104,1


In [65]:
knnbasic.predict(uid='U1068',iid='135104')

Prediction(uid='U1068', iid='135104', r_ui=None, est=1.1998277347114557, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'})

In [66]:
#actual = 1 Predicted = 1.19

In [67]:
## instead of using just one train set, we can split the data in 3 folds and then evaluate the performance out of sample

In [92]:
from surprise.model_selection import KFold
kf = KFold(n_splits=3)
algo = SVD()

In [106]:
from surprise import SVD
from surprise import accuracy
from surprise import model_selection
from surprise.model_selection import GridSearchCV


In [113]:
from surprise.model_selection import cross_validate
cross_validate(knnbasic, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
# out of sample performance of this model is measured by root mean squared error

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8054  0.7784  0.7888  0.7909  0.0111  
MAE (testset)     0.6522  0.6258  0.6335  0.6372  0.0111  
Fit time          0.00    0.00    0.00    0.00    0.00    
Test time         0.02    0.01    0.01    0.02    0.00    


{'test_rmse': array([0.80544367, 0.77841577, 0.78878129]),
 'test_mae': array([0.65224902, 0.62580947, 0.63349416]),
 'fit_time': (0.0019986629486083984,
  0.001995086669921875,
  0.0019931793212890625),
 'test_time': (0.01894235610961914, 0.013993501663208008, 0.01396322250366211)}

In [132]:
from surprise.model_selection import GridSearchCV

#knnmeans = knns.KNNWithMeans(k=40,sims_options={'name':'pearson','user_based':False})

param_grid = {'k': [10, 20, 30], 'sims_options': {'name':['msd','cosine'],'user_based':False}}

grid_search = GridSearchCV(knns.KNNWithMeans,param_grid, measures=['RMSE','MAE'], cv=3)
grid_search.fit(data)

print(grid_search.best_score['rmse'])
print(grid_search.best_score['mae'])
print(grid_search.best_params['rmse'])
print(grid_search.best_params['mae'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [136]:
results_df = pd.DataFrame.from_dict(grid_search.cv_results)
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_k,param_sims_options
0,0.748317,0.738834,0.727346,0.738166,0.008575,5,0.584667,0.587259,0.578284,0.583403,0.003771,5,0.005316,0.000471,0.015956,0.009177,"{'k': 10, 'sims_options': 'name'}",10,name
1,0.748317,0.738834,0.727346,0.738166,0.008575,6,0.584667,0.587259,0.578284,0.583403,0.003771,6,0.004986,0.002936,0.010316,0.002049,"{'k': 10, 'sims_options': 'user_based'}",10,user_based
2,0.747968,0.738574,0.727049,0.737864,0.008555,3,0.584908,0.586939,0.578211,0.583352,0.003729,3,0.003647,0.000485,0.010317,0.000487,"{'k': 20, 'sims_options': 'name'}",20,name
3,0.747968,0.738574,0.727049,0.737864,0.008555,4,0.584908,0.586939,0.578211,0.583352,0.003729,4,0.003647,0.000927,0.009313,0.002054,"{'k': 20, 'sims_options': 'user_based'}",20,user_based
4,0.747917,0.738574,0.727049,0.737847,0.008535,1,0.584848,0.586939,0.578211,0.583332,0.003721,1,0.004985,0.000813,0.010319,0.003088,"{'k': 30, 'sims_options': 'name'}",30,name
5,0.747917,0.738574,0.727049,0.737847,0.008535,2,0.584848,0.586939,0.578211,0.583332,0.003721,2,0.003324,0.000469,0.008416,0.001551,"{'k': 30, 'sims_options': 'user_based'}",30,user_based


In [139]:
## Top 5 recommendation for an item
model = knns.KNNWithMeans(k=30, sims_options = {'name':'msd','user_based':False})


In [141]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [142]:
top_n = get_top_n(predictions, n=10)

In [143]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

U1134 [135075, 135044, 135085, 135047, 135074, 135064, 135059, 135079, 132572, 135080]
U1109 [135085, 135042, 132825, 132872]
U1083 [135074, 132572]
U1135 [135060, 135052, 132856, 132825, 135042]
U1041 [135001, 134983, 135109, 135021]
U1014 [135076, 135081, 135050, 135069]
U1104 [135047, 135085, 132825, 135039, 135041, 135033]
U1082 [132732, 132560, 132630, 132608]
U1013 [132921, 132875, 135076, 135079, 135060]
U1105 [135040]
U1001 [135045, 135040, 132830]
U1024 [135039, 135073, 135106, 135038]
U1009 [132825, 135059, 135058]
U1039 [132626, 132564, 132665]
U1117 [132773, 132766]
U1100 [132768, 135000, 135021]
U1026 [132626, 132561, 132706, 132665, 132654]
U1047 [132884, 132875]
U1089 [132951, 132875, 132921, 132834, 135064, 135079]
U1022 [135032, 135045, 135062, 135040, 135038, 135081]
U1037 [135082, 135071, 132847, 135070]
U1068 [132630, 132663, 135104]
U1075 [135039, 135066, 135032, 135050]
U1099 [135043, 132872]
U1029 [135085, 132754, 132825, 132937, 132921]
U1123 [132594, 132667, 13